## Agent Framework Workflows Orchestrations
Traditional single-agent systems are limited in their ability to handle complex, multi-faceted tasks. 

Multi-agent orchestration addresses this limitation by allowing you to:

- Assign distinct skills, responsibilities, or perspectives to each agent.
- Combine outputs from multiple agents to improve decision-making and accuracy.
- Coordinate steps in a workflow so each agent’s work builds on the last.
- Dynamically route control between agents based on context or rules.

**Orchestrations** are pre-built workflow patterns that allow developers to quickly create complex workflows by simply plugging in their own AI agents. Instead of manually wiring every edge, you can use these patterns to define how agents interact.

Because all patterns share the same core interface, you can easily experiment with different orchestration strategies without rewriting agent logic or learning new APIs. The SDK abstracts the complexity of agent communication, coordination, and result aggregation so you can focus on designing workflows that deliver results.

Agent Framework provides four orchestration patterns:

1. **Sequential** - Agents execute one after another in a defined order, ideal for linear processing
2. **Concurrent** - Broadcast the same task to multiple agents at once and collect results independently.
3. **Handoff** - Control is dynamically transferred from one agent to another based on context or role.
4. **Group chat** - Coordinate a shared conversation among multiple agents managed by a chat manager that decides who speaks next and when to stop.
4. **Magentic** - Manager-driven approach where agents dynamically pull tasks based on the evolving context, task progress, and agent capabilities.

Refer to the [Agent Framework documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/overview) to learn more about these orchestration types and check out the [Agent Framework repo](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/workflows/orchestration) to see more advanced orchestration types in action.

In this following sections, we’ll introduce Sequential, Concurrent and Group Chat orchestrations:

In [ ]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

load_dotenv()
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

chat_client=AzureOpenAIChatClient(
        endpoint=endpoint,
        api_key=api_key,
        api_version=api_version,
        deployment_name=deployment,
    )

### Example of Sequential Orchestration
In sequential orchestration, agents are organized in a pipeline. Each agent processes the task in turn, passing its output to the next agent in the sequence. This is ideal for workflows where each step builds upon the previous one, such as document review, data processing pipelines, or multi-stage reasoning.

The `SequentialBuilder` class creates a pipeline where agents process tasks in order. Each agent sees the full conversation history and adds their response.

In [ ]:
from typing import Any
from agent_framework import SequentialBuilder, ChatMessage, Role

# 1) Create agents from chat_client:
meal_planner = chat_client.create_agent(
    instructions=(
        "You are a meal planner. Suggest a simple, healthy weekly meal plan for breakfast, lunch, and dinner."
    ),
    name="meal_planner",
)

budget_estimator = chat_client.create_agent(
    instructions=(
        "You are a budget estimator. Calculate an approximate total cost for the 7-day meal plan provided by the previous assistant."
    ),
    name="budget_estimator",
)

# 2) Build sequential workflow: meal_planner -> budget_estimator
workflow = SequentialBuilder().participants([meal_planner, budget_estimator]).build()

# 3) Run and collect messages
messages: list[ChatMessage] = []
async for event in workflow.run_stream("Create a short weekly meal plan for one person."):
    if hasattr(event, "data") and isinstance(event.data, list):
        messages.extend(event.data)

# 4) Print final conversation
print("===== Final Conversation =====")
for i, msg in enumerate(messages, start=1):
    name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
    print(f"{'-' * 60}\n{i:02d} [{name}]\n{msg.text}")

## Concurrent Orchestration
Concurrent orchestration enables multiple agents to work on the same task in parallel. Each agent processes the input independently, and their results are collected and aggregated. This approach is well-suited for scenarios where diverse perspectives or solutions are valuable. 
The `ConcurrentBuilder` class allows you to construct a workflow that runs multiple agents in parallel.

- You pass a list of agents as participants.
- The builder automatically orchestrates their execution concurrently.
- Results are returned as a collection of outputs from all agents.

The following example demonstrates how to:

1. Build a concurrent workflow orchestration using `ConcurrentBuilder`
2. Fan-out to multiple agents, fan-in aggregation of final ChatMessages.
3. Expose the entire workflow as a reusable agent via `workflow.as_agent(...)`, allowing downstream workflows or coordinators to treat the orchestration as a single agent entry point.
4. Handle workflow completion when all parallel tasks finish.

In [ ]:
from agent_framework import ConcurrentBuilder, ChatMessage

async def main() -> None:
    # 1) Create three domain agents using AzureOpenAIChatClient

    researcher = chat_client.create_agent(
        instructions=(
            "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
            " opportunities, and risks."
        ),
        name="researcher",
    )

    marketer = chat_client.create_agent(
        instructions=(
            "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
            " aligned to the prompt."
        ),
        name="marketer",
    )

    legal = chat_client.create_agent(
        instructions=(
            "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
            " based on the prompt."
        ),
        name="legal",
    )

    # 2) Build a concurrent workflow
    workflow = ConcurrentBuilder().participants([researcher, marketer, legal]).build()


    prompt = "We are launching a new budget-friendly electric bike for urban commuters."
    agent_response = await workflow.run(prompt)

    
    # Add user input as first message
    outputs = agent_response.get_outputs()

    if outputs:
            print("===== Final Aggregated Conversation (messages) =====")
            for output in outputs:
                messages: list[ChatMessage] | Any = output
                for i, msg in enumerate(messages, start=1):
                    name = msg.author_name if msg.author_name else "user"
                    print(f"{'-' * 60}\n\n{i:02d} [{name}]:\n{msg.text}")


await main()

## Group Chat Orchestration

**Group chat orchestration** models a collaborative conversation among multiple AI agents and optionally a human participant. A central chat manager controls the flow and decides which agent responds next or when to request human input.

Group chat is ideal for iterative refinement, debate, and multi‑perspective problem solving where agents share the same conversation context and can build on each other’s outputs.

**Core parts of Group chat orchestration:**
1. **Agents**

Individual ChatAgents with distinct roles (e.g., Planner, Engineer, Reviewer). Each agent can reason independently but collaborates through the orchestrator.

2. `GroupChatBuilder()`

Fluent API to assemble a group chat with participants, speaker selection strategy or a prompt‑based manager.

3. `GroupChatStateSnapshot`
    - Immutable snapshot that contains the following:
        - task: `ChatMessage` - original user task
        - participants: `dict[str, str]` - participant names → descriptions
        - conversation: `tuple[ChatMessage, ...]` - full conversation history
        - history: `tuple[GroupChatTurn, ...]` - turn-by-turn with speaker attribution
        - round_index: `int` - number of selection rounds so far
        - pending_agent: `str | None` - currently active agent (if any)

You can use this information to define a custom selection logic and implement guardrails.

4. **Selection Strategy**

The policy that decides who speaks next each turn. You can choose between deterministic code‑based selection or use the default AI‑managed selection. The latter uses the default group chat orchestration pipeline shared with Magentic, so you benefit from established patterns for iterative collaboration and stopping criteria without writing orchestration glue.
To add an AI-powered manager, just provide the chat client in the workflow definition:
```python
    workflow = (
        GroupChatBuilder()
        .set_prompt_based_manager(chat_client=AzureOpenAIChatClient(), display_name="Orchestrator")
        .participants(researcher=researcher, writer=writer)
        .build()
    )
```

### Example - Group Chat with Custom Speaker Selection

In the following example we'll implement a three agent group chat with a simple next speaker selector.
Key Objectives:
- Implement a deterministic `select_speakers()` tool that takes a `GroupChatStateSnapshot` and returns the next participant.
- Understand the state fields you can use for decisions: task, participants, conversation, history, round_index, pending_agent.
- Event‑driven observability with streaming, event handling and middleware.

First, let's create and provide detailed instructions for the agents:

In [ ]:
from agent_framework import agent_middleware

@agent_middleware 
async def logging_agent_middleware(context, next):  
    """Agent middleware with decorator - types are inferred."""
    print(f"[{context.agent.name} Agent] INVOKING")
    await next(context)   
    print(f"[{context.agent.name} Agent] FINISHED")

writer = chat_client.create_agent(
    name="Writer",
    instructions="""
    You are a creative writer crafting engaging content.

    Role:
    1. Generate original, compelling narratives.
    2. Apply storytelling techniques for engagement.
    3. Incorporate feedback from editor/researcher.
    4. Revise to address issues while preserving core message.

    Feedback:
    - Accept constructive criticism, Explain creative choices when needed, Keep responses concise, imaginative, and engaging.""",
    middleware=[logging_agent_middleware]
)

editor = chat_client.create_agent(
    name="Editor",
    instructions="""
    You are an editor focused on clarity and quality.

    Role:
    1. Ensure clarity, coherence, and flow.
    2. Fix grammar, structure, and style.
    3. Suggest improvements for readability and impact.

    Feedback:
    - Be specific and constructive, Address sentence-level and structural issues, Respect writer's voice while improving content.
    Keep responses concise and clear.""",
    middleware=[logging_agent_middleware]
)


researcher = chat_client.create_agent(
    name="Researcher",
    instructions="""
    You provide accurate, relevant research.

    Role:
    1. Gather credible, up-to-date info.
    2. Summarize findings clearly.
    3. Highlight key insights and fill gaps.
    4. Ensure factual accuracy and alignment with goals.

    Responses:
    - Base on verifiable facts, Use bullet points or short paragraphs, Suggest sources when useful.
    Keep responses concise, relevant, and actionable.""",
    middleware=[logging_agent_middleware]
)


In [ ]:
import logging

from agent_framework import (
    ChatMessage, 
    GroupChatBuilder,
    GroupChatStateSnapshot,
    WorkflowOutputEvent, 
    AgentRunUpdateEvent
)

logging.basicConfig(level=logging.INFO)

def select_next_speaker(state: GroupChatStateSnapshot) -> str | None:

    participants = state["participants"]             
    history = state["history"]
    round_idx = state["round_index"]

    # Construct the effective cycle from the preferred order, keeping only roles that are actually present.
    preferred = ["Researcher", "Writer", "Editor"]
    order = [p for p in preferred if p in participants]

    # Finish after 6 turns (researcher -> writer -> editor -> researcher -> writer)
    if round_idx >= 6:
        return None

    # First turn: start with the first available in the preferred order
    if not history:
        return order[0]

    last = history[-1].speaker

    # If last speaker is not recognized (e.g., system/user), restart from the first
    try:
        i = order.index(last)
        return order[(i + 1) % len(order)]
    except ValueError:
        return order[0]


workflow = (
    GroupChatBuilder()
    .select_speakers(select_next_speaker, display_name="Orchestrator")
    .participants([researcher, writer, editor])  # Uses agent.name for participant names
    .build()
)

task = "What are the key benefits of using async/await in Python?"

print("\nStarting Group Chat with Simple Speaker Selector...\n")
print(f"TASK: {task}\n")
print("=" * 80)


async for event in workflow.run_stream(
    ChatMessage(role="user", text=task)
):
    if isinstance(event, AgentRunUpdateEvent):
        print(f"{event.data}", end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        print(f"Workflow output ({event.origin.value}): {event.data}")

print("\nWorkflow completed.")

 **Note:**
In the output, you might notice that middleware logs FINISHED before the agent-generated content appears. 

In streaming mode, the agent returns an async generator, so the FINISHED log simply means the generator was created and handed back, not that all tokens have been printed. The actual text follows via `AgentRunUpdateEvent` as tokens stream progressively.

### Exercise - create your own orchestration pattern scenario

Now that you’ve explored and tested a few orchestration examples, let's apply what you’ve learned. Think about a real-world domain or business logic case, such as customer support, reporting, knowledge work automation or any process relevant to your context. Your goal is to implement your own orchestration that reflects this scenario using the concepts and patterns introduced earlier.

**Typical flow for building a multi-agent orchestration**
1. Clearly define your agents and describe their roles and capabilities.
2. Select an orchestration pattern and create it, optionally adding a manager agent for coordination.
3. Configure callbacks or transforms if you need custom input/output handling.
4. Start a runtime to manage execution.
5. Invoke the orchestration with your chosen task.
6. Retrieve results asynchronously for non-blocking, real-time feedback.

Alternatively, you can also experiment with advanced patterns like Handoff or Magentic orchestration which we haven’t covered. Feel free to explore these approaches using the additional documentation and resources:
- [Handoff Documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/handoff?pivots=programming-language-python)
- [Github Handoff Samples](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/orchestration/handoff_simple.py)
- [Magentic Docs](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/magentic?pivots=programming-language-python)
- [Magentic Samples](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/magentic_workflow_as_agent.py)



In [ ]:
# Your implementation here

# 1. Define your agents: roles & capabilities - describe responsibilities, constraints, tools, and style. Keep instructions specific.

# 2. Consider and define any function tools that might be useful

# 3. Select & create an orchestration pattern

# 4. Configure callbacks/transforms, add executors and consider obsevability 

# 5. Define and invoke the orchestration with your chosen task

# 6. Retrieve results, leverage the event types

# (Optional) Stress-test with at least two different tasks and compare outcome

### Choosing an Orchestration

When selecting an orchestration pattern, try to think about this:
1. **Group chat** is best for:
    - Spontaneous or guided collaboration among agents (and possibly humans)
    - Iterative maker-checker loops where agents take turns creating and reviewing
    - Real-time human oversight or participation
    - Transparent and auditable conversations since all output is collected in a single thread
2. **Sequential** for:
    - Data workflows where each stage adds something important that the next stage needs to work properly.
    - Situations that require gradual improvements, like drafting, reviewing, and polishing content.
    - Systems where you know how each agent performs and can handle delays or failures in any step without stopping the whole process.
3. **Concurrent** for:
    - When tasks can run at the same time, either by using a fixed group of agents or by selecting AI agents dynamically based on what the task needs.
    - When the task benefits from different specialized skills or approaches (for example, technical, business, or creative) that all work independently but contribute to solving the same problem.

4. **Handoff**:
    - Tasks need specialized knowledge or tools, but the number or order of agents can't be determined in advance.
    - Expertise requirements emerge dynamically during processing, triggering task routing based on content analysis.
    - Multiple-domain problems require different specialists working sequentially.